In [ ]:
!pip install neo4j



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase

# Sandbox details (use your values here 👇)
URI = "bolt://13.220.157.217:7687"
AUTH = ("neo4j", "lace-typist-comment")

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session(database="neo4j") as session:
    result = session.run("""
        MATCH path = (start:Task)-[:DEPENDS_ON*]->(end:Task)
        WHERE NOT ()-[:DEPENDS_ON]->(start)
          AND NOT (end)-[:DEPENDS_ON]->()
        WITH path,
             REDUCE(total = 0, t IN nodes(path) | total + t.duration) AS path_duration
        ORDER BY path_duration DESC
        LIMIT 1
        RETURN [t IN nodes(path) | t.name] AS critical_path, path_duration
    """)

    for record in result:
        print("Critical Path:", record["critical_path"])
        print("Total Duration:", record["path_duration"], "days")



Critical Path: ['Deployment', 'Testing', 'Model Integration', 'Data Pipeline']
Total Duration: 11 days
